# Обработка исключений

Философия. Пишем программу, не думая о том, что может произойти исключительная ситуация. (ошибка). Например, деление на ноль, выход за границу списка, ошибка чтения файла и т.п. Т.е. программируем оптимистичный сценарий выполнения программы. Но, дополнительно, отдельно от основной программы, пишем код, который нужно выполнить, если ошибка все-таки случилась:

In [1]:
try:
    1 // 0  # гарантированная ошибка
    print("пронесло")
except:
    print("ой")

ой


Если ошибки не произошло, то блок `try` выполняется полностью, блок `except` не выполняется вообще.

Но если ошибка случилась, выполнение блока `try` прекращается, и управление передается в `except`.

In [2]:
x = 42
try:
    1 // x  # ошибка, если x = 0
    print("пронесло")
except:
    print("ой")

пронесло


Блок `finally` можно добавить, код в нем исполняется гарантированно (100% гарантии), независимо от того, была ли ошибка. Если есть `finally`, то можно не делать `except`:

In [3]:
def some_function(x):
    try:
        1 // x  # ошибка, если x = 0
        print("пронесло")
    except:
        print("ой")
    finally:
        print("точно напечатается")
        
some_function(42)
print('------------')
some_function(0)

пронесло
точно напечатается
------------
ой
точно напечатается


После выполнения блока `except` ошибка считается обработанной, и выполнение кода переходит к тому, что написано после `try-except-finally`:

In [4]:
try:
    1 // 0
    print("не печатается из-за того, что в предыдущей строке ошибка")
except:
    print("ой")

print("продолжение кода")  # продолжает выполняться после 

ой
продолжение кода


Что происходит с необработанной ошибкой?

In [5]:
1 // 0

ZeroDivisionError: integer division or modulo by zero

Появляется стандартный текст с сообщением о типе ошибке (здесь: `ZeroDivisionError`),
человеко-читаемом сообщении об ошибке (здесь: `integer division or modulo by zero`) и
информация о том, где случилась ошибка. В обычном python, не в jupyter-блокнотах, информация представлена более сжато.

## Стек вызовов
Что происходит с необработанной ошибкой, если она появляется при вызове функции:

In [6]:
def danger():
    [][0]  # гарантированная ошибка, в пустом списке нет нулевого элемента
    print(42)
    
def g():
    danger()
    print(42)
    
def f():
    g()
    print(42)
    
f()
print(42)

IndexError: list index out of range

Итого, если в функции ошибка не обрабатывается в блоке `except`, то считается, что вызов этой функции привел к ошибке. Код после ошибки не выполняется.

In [7]:
try:
    f()
    print(42)
except:
    print("какая-то ошибка")
    
print(123)

какая-то ошибка
123


Если вы используете какую-то функцию, обычно про нее известно, к каким ошибкам она может привести. Об этом написано в ее документации. При необходимости, вы можете обернуть вызов в блок `try`, чтобы обработать ошибку.

Если вызов функции может привести к ошибке, вы должны решить, обработать ее сразу в блоке `catch` или не обрабатывать, но тогда ответственность за обработку переходит к вызывающей функции.

## Иерархия исключений

Ошибки (исключения) бывают разные: `ZeroDivisionError`, `IndexError` и много других. Все ошибки образуют иерархию. Например, `ZeroDivisionError` - это частный случай `ArithmeticError`, и `OverflowError` тоже частный случай `ArithmeticError`. А `ArithmeticError` частный случай `Exception`, а `Exception` - частный случай `BaseException`. Все ошибки - это частные случаи `BaseException`.

Можно делать несколько блоков `except` для разных видов ошибок. Они проверяются по-очереди:

In [8]:
try:
    1 // 0  # ZeroDivisionError
except OverflowError:
    print("что-то переполнилось")
except ArithmeticError:
    print("что-то плохо посчиталось")
except ZeroDivisionError:  # никогда не выполнится, потому что перед этим сработает ArithmeticError
    print("поделили на ноль")
except:  # аналогично except BaseException, т.е. ловится все подряд
    print("что-то плохо посчиталось")

что-то плохо посчиталось


`Exception` - это частный случай почти всех исключений, обычно можно рассчитывать, что в вашей программе происходят только они. `BaseException` содержит исключения типа `SystemExit`, такое не нужно обрабатывать.

Если вы будете сами создавать свой тип исключений, сделайте его частным случаем `Exception`. Как? См. наследование из ООП.

## Генерация исключения
Оператор `raise`:

In [9]:
raise IndexError("какой-то неправильный индекс")  # создали исключение и вызывали его

IndexError: какой-то неправильный индекс

## Информация об исключении
При обработке исключения можно узнать о нем информацию:

In [10]:
try:
    1 // 0
except ArithmeticError as e:  # as e
    print(e)

integer division or modulo by zero


Переменная `e` содержит информацию об ошибке. Она находится в кортеже `e.args`, чаще всего там есть только один текст с сообщением, что именно случилось.